In [1]:
import sys
sys.path.append('/Users/mask/Documents/Проекты_2026/wb_analiz')

from wb_api import get_orders, get_stocks, merge_orders_stocks, calc_avg_per_day
import pandas as pd

# "Мы берём всех товаров и смотрим: топ-25% продают больше 4 штук в день — это ходовые. Нижние 25% продают меньше 0.5 штуки в день — это редкие."
# Пороги для групп (шт/день, на основе 30-дневных данных)
THRESHOLD_A = 4.0   # A: ≥4
THRESHOLD_B = 0.5   # B: ≥0.5, C: <0.5

In [2]:
# Получаем данные за 30 дней
orders_30d = get_orders(30)
stocks = get_stocks(orders_30d['nmId'].tolist())

# Объединяем и считаем среднее
df = merge_orders_stocks(orders_30d, stocks)
df = calc_avg_per_day(df, days=30)

print(f'Загружено товаров: {len(df)}')
df.head()

Загружено товаров: 137


,nmId,supplierArticle,subject,category,orders_count_30d,stock_qty,avg_per_day_30d
0,198215260,7*9бел20,Мешочки подарочные,Для праздника,3032,1987,101.066667
1,185297285,10*12бел20,Мешочки подарочные,Для праздника,1706,1577,56.866667
2,460927109,крестмал,Цепочки,Бижутерия,4984,557,166.133333
3,351172702,кноп18сер10,Аксессуары для рукоделия,Рукоделие,481,478,16.033333
4,351163316,кноп14сер10,Аксессуары для рукоделия,Рукоделие,855,462,28.500000


In [3]:
def assign_group(avg_per_day: float) -> str:
    """Определяет группу товара по средним продажам в день."""
    if avg_per_day >= THRESHOLD_A:
        return 'A'
    elif avg_per_day >= THRESHOLD_B:
        return 'B'
    else:
        return 'C'

df['group'] = df['avg_per_day_30d'].apply(assign_group)

# Проверяем распределение
print(df['group'].value_counts().sort_index())

group
A    36
B    47
C    54
Name: count, dtype: int64


In [4]:
# Для группы A — 7 дней
orders_7d = get_orders(7)
orders_7d = orders_7d[['nmId', 'orders_count_7d']]

# Для группы B — 14 дней
orders_14d = get_orders(14)
orders_14d = orders_14d[['nmId', 'orders_count_14d']]

# Присоединяем к основной таблице
df = df.merge(orders_7d, on='nmId', how='left')
df = df.merge(orders_14d, on='nmId', how='left')

print(f'Колонки: {df.columns.tolist()}')
df.head()

Колонки: ['nmId', 'supplierArticle', 'subject', 'category', 'orders_count_30d', 'stock_qty', 'avg_per_day_30d', 'group', 'orders_count_7d', 'orders_count_14d']


,nmId,supplierArticle,subject,category,orders_count_30d,stock_qty,avg_per_day_30d,group,orders_count_7d,orders_count_14d
0,198215260,7*9бел20,Мешочки подарочные,Для праздника,3032,1987,101.066667,A,932.0,1592.0
1,185297285,10*12бел20,Мешочки подарочные,Для праздника,1706,1577,56.866667,A,420.0,808.0
2,460927109,крестмал,Цепочки,Бижутерия,4984,557,166.133333,A,842.0,1832.0
3,351172702,кноп18сер10,Аксессуары для рукоделия,Рукоделие,481,478,16.033333,A,138.0,199.0
4,351163316,кноп14сер10,Аксессуары для рукоделия,Рукоделие,855,462,28.500000,A,224.0,363.0


In [5]:
def calc_avg_by_group(row) -> float:
    """Возвращает среднее в день по периоду группы."""
    if row['group'] == 'A':
        return row['orders_count_7d'] / 7 if pd.notna(row['orders_count_7d']) else 0
    elif row['group'] == 'B':
        return row['orders_count_14d'] / 14 if pd.notna(row['orders_count_14d']) else 0
    else:  # C
        return row['avg_per_day_30d']

df['avg_per_day'] = df.apply(calc_avg_by_group, axis=1)

df[['nmId', 'supplierArticle', 'group', 'avg_per_day']].head(10)

,nmId,supplierArticle,group,avg_per_day
0,198215260,7*9бел20,A,133.142857
1,185297285,10*12бел20,A,60.000000
2,460927109,крестмал,A,120.285714
3,351172702,кноп18сер10,A,19.714286
4,351163316,кноп14сер10,A,32.000000
5,210784687,торт1,A,24.000000
6,200189009,аниме3,A,6.428571
7,198215262,7*9бел50,A,8.000000
8,201451087,сум1кр,A,17.714286
9,197759511,7*9микс20,A,8.428571


In [6]:
def calc_days_remaining(row) -> float:
    """На сколько дней хватит остатка."""
    if row['avg_per_day'] == 0:
        return None  # нет продаж — неизвестно
    return row['stock_qty'] / row['avg_per_day']

df['days_remaining'] = df.apply(calc_days_remaining, axis=1)

df[['nmId', 'supplierArticle', 'group', 'stock_qty', 'avg_per_day', 'days_remaining']].head(10)

,nmId,supplierArticle,group,stock_qty,avg_per_day,days_remaining
0,198215260,7*9бел20,A,1987,133.142857,14.923820
1,185297285,10*12бел20,A,1577,60.000000,26.283333
2,460927109,крестмал,A,557,120.285714,4.630641
3,351172702,кноп18сер10,A,478,19.714286,24.246377
4,351163316,кноп14сер10,A,462,32.000000,14.437500
5,210784687,торт1,A,432,24.000000,18.000000
6,200189009,аниме3,A,369,6.428571,57.400000
7,198215262,7*9бел50,A,327,8.000000,40.875000
8,201451087,сум1кр,A,310,17.714286,17.500000
9,197759511,7*9микс20,A,299,8.428571,35.474576


In [7]:
# опционально: смотрим всю таблицу
pd.set_option('display.max_rows', 150)

df_sorted = df.sort_values(['group', 'days_remaining'], ascending=[True, True])
df_sorted[['nmId', 'supplierArticle', 'group', 'stock_qty', 'avg_per_day', 'days_remaining']]

,nmId,supplierArticle,group,stock_qty,avg_per_day,days_remaining
94,151830787,полка в ванную белая,A,0,1.285714,0.000000
98,256012121,Кноп18зол10,A,0,13.000000,0.000000
103,585059945,лофтспеции2,A,0,8.285714,0.000000
104,210114872,сум10кр,A,0,8.142857,0.000000
119,404393669,10*12черн20,A,0,1.857143,0.000000
127,275089247,Сум8,A,0,2.000000,0.000000
90,201451089,сум4,A,1,6.000000,0.166667
82,156778907,виз1н,A,4,20.285714,0.197183
84,404398825,15*20бел10,A,3,4.571429,0.656250
72,437752932,кноп18чер10,A,15,20.142857,0.744681


In [11]:
def get_price_increase(days_remaining: float) -> int:
    """Возвращает % повышения цены по шкале."""
    if days_remaining is None or days_remaining > 7:
        return 0
    elif days_remaining >= 6:  # 6 ≤ x ≤ 7
        return 5
    elif days_remaining >= 5:  # 5 ≤ x < 6
        return 10
    elif days_remaining >= 4:  # 4 ≤ x < 5
        return 15
    elif days_remaining >= 3:  # 3 ≤ x < 4
        return 25
    elif days_remaining >= 2:  # 2 ≤ x < 3
        return 30
    elif days_remaining >= 1:  # 1 ≤ x < 2
        return 40
    else:                      # x < 1
        return 50

df['price_increase_pct'] = df['days_remaining'].apply(get_price_increase)

# Проверяем распределение
print(df['price_increase_pct'].value_counts().sort_index())

price_increase_pct
0     73
10     3
15     3
25     1
30     2
40     4
50    51
Name: count, dtype: int64


In [13]:
# Фильтруем: есть остаток И нужно повышение
report = df[(df['stock_qty'] > 0) & (df['price_increase_pct'] > 0)].copy()
report = report.sort_values('days_remaining')

report[['supplierArticle', 'group', 'stock_qty', 'avg_per_day', 'days_remaining', 'price_increase_pct']]

,supplierArticle,group,stock_qty,avg_per_day,days_remaining,price_increase_pct
90,сум4,A,1,6.000000,0.166667,50
82,виз1н,A,4,20.285714,0.197183,50
86,сум1роз,B,2,4.571429,0.437500,50
84,15*20бел10,A,3,4.571429,0.656250,50
72,кноп18чер10,A,15,20.142857,0.744681,50
79,кноп10сер10,A,6,6.000000,1.000000,40
75,пласт4тонк,B,8,6.285714,1.272727,40
87,чехол для ручки коричневый 10 штук,B,2,1.500000,1.333333,40
65,бархат1,A,21,11.428571,1.837500,40
78,виз6к,B,7,3.285714,2.130435,30


In [10]:
# Товары с нулевым остатком
out_of_stock = df[df['stock_qty'] == 0].copy()
out_of_stock = out_of_stock.sort_values(['group', 'avg_per_day'], ascending=[True, False])

print(f'Товаров с нулевым остатком: {len(out_of_stock)}')
print(f'Упущенные продажи в день: {out_of_stock["avg_per_day"].sum():.1f} шт')

out_of_stock[['supplierArticle', 'group', 'avg_per_day']]

Товаров с нулевым остатком: 46
Упущенные продажи в день: 63.7 шт


,supplierArticle,group,avg_per_day
98,Кноп18зол10,A,13.000000
103,лофтспеции2,A,8.285714
104,сум10кр,A,8.142857
127,Сум8,A,2.000000
119,10*12черн20,A,1.857143
94,полка в ванную белая,A,1.285714
117,торткоробка,B,4.714286
110,напяточниксилик,B,4.071429
97,лофт7.5*37,B,3.500000
115,цен50,B,2.142857


In [14]:
output_path = '/Users/mask/Documents/Проекты_2026/wb_analiz/price_report.xlsx'

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    # Лист 1: Товары для повышения цены
    report_export = report[['supplierArticle', 'group', 'stock_qty', 'avg_per_day', 'days_remaining', 'price_increase_pct']]
    report_export.to_excel(writer, sheet_name='Повысить цену', index=False)
    
    # Лист 2: Упущенные продажи
    out_of_stock_export = out_of_stock[['supplierArticle', 'group', 'avg_per_day']]
    out_of_stock_export.to_excel(writer, sheet_name='Нет на складе', index=False)
    
    # Добавляем аннотации внизу
    ws = writer.sheets['Нет на складе']
    last_row = len(out_of_stock_export) + 3
    ws.cell(row=last_row, column=1, value=f'Товаров с нулевым остатком: {len(out_of_stock)}')
    ws.cell(row=last_row + 1, column=1, value=f'Упущенные продажи в день: {out_of_stock["avg_per_day"].sum():.1f} шт')

print(f'Сохранено: {output_path}')

Сохранено: /Users/mask/Documents/Проекты_2026/wb_analiz/price_report.xlsx
